In [1]:
import sys

sys.path.append('D:\PythonProjects\\bk-monitor\\venv2\share\scripts')
import hello
from django.conf import settings
from bkmonitor.models import StrategyModel, ItemModel
from core.drf_resource import api, resource
import time

settings.ENABLED_TARGET_CACHE_BK_BIZ_IDS = [2] * 100

当前运行环境缺少部分需要的环境变量，可能导致项目运行报错，请确认以下环境变量已经配置并赋值: BK_PAAS2_URL
注意，当前版本为社区版环境，请确认 BKPAAS_MAJOR_VERSION 已经配置并正确赋值


In [2]:
def update_target_detail():
    """
    对启用了缓存的业务ID，更新监控目标详情缓存
    """
    for bk_biz_id in settings.ENABLED_TARGET_CACHE_BK_BIZ_IDS:
        strategy_ids = StrategyModel.objects.filter(bk_biz_id=bk_biz_id).values_list("id", flat=True)
        items = ItemModel.objects.filter(strategy_id__in=strategy_ids).only("strategy_id", "target")
        mapping = {item.strategy_id: (bk_biz_id, item.target) for item in items}
        resource.strategies.get_target_detail_with_cache.set_mapping(mapping)
        for item in items:
            try:
                resource.strategies.get_target_detail_with_cache.request.refresh({"strategy_id": item.strategy_id})
            except Exception as e:
                # logger.exception(f"Update target detail cache failed for strategy id [{item.strategy_id}]: {e}")
                print(f"Update target detail cache failed for strategy id [{item.strategy_id}]: {e}")

old = time.time()
update_target_detail()
print(f"Update target detail cache took {time.time() - old}")

In [9]:
from concurrent.futures import ThreadPoolExecutor
import traceback


def update_target_detail():
    """
    对启用了缓存的业务ID，更新监控目标详情缓存
    """

    def inner(bk_biz_id):
        strategy_ids = StrategyModel.objects.filter(bk_biz_id=bk_biz_id).values_list("id", flat=True)
        items = ItemModel.objects.filter(strategy_id__in=strategy_ids).only("strategy_id", "target")
        resource.strategies.get_target_detail_with_cache.set_mapping({item.strategy_id: (bk_biz_id, item.target)
                                                                      for item in items})
        for item in items:
            try:
                resource.strategies.get_target_detail_with_cache.request.refresh({"strategy_id": item.strategy_id})
            except Exception as e:
                # logger.exception(f"Update target detail cache failed for strategy id [{item.strategy_id}]: {e}")
                traceback.print_exc()
                print(f"Update target detail cache failed for strategy id [{item.strategy_id}]: {e}")

    with ThreadPoolExecutor() as executor:

        for bk_biz_id in settings.ENABLED_TARGET_CACHE_BK_BIZ_IDS:
            executor.submit(inner, bk_biz_id)

old = time.time()
update_target_detail()
print(f"Update target detail cache took {time.time() - old}")

Update target detail cache took 6.334095478057861
